!pip install dateparser

In [ ]:
import random
import json
import csv
import numpy as np
import pandas as pd
import datetime
import dateparser

import requests
from requests.auth import HTTPBasicAuth


%load_ext dotenv
#%reload_ext dotenv

# Use find_dotenv to locate the file
%dotenv
import os

In [ ]:
HOST = "http://localhost:5000"

# auto daily report (flex query)

In [ ]:
ib = {
    'QUERY_ID': '332646',
    'TOKEN': os.getenv('TOKEN')
}


res = requests.post(
    f'{HOST}/reports/ib/eod/v2', 
    json=ib, 
    auth=HTTPBasicAuth(os.getenv('BASIC_AUTH_USERNAME'), os.getenv('BASIC_AUTH_PASSWORD')))
json.loads(res.content)

In [ ]:
ib = {
    'QUERY_ID': '332646',
    'TOKEN': os.getenv('TOKEN')
}

res = requests.get(
    f'{HOST}/reports/ib/eod/v2/reportDate', 
    json=ib, 
    auth=HTTPBasicAuth(os.getenv('BASIC_AUTH_USERNAME'), os.getenv('BASIC_AUTH_PASSWORD')))
json.loads(res.content)

# xls fa_data

In [ ]:
res = requests.get(
    f'{HOST}/reports/ib/eod/v2', 
#    json=json.loads(requests.get('http://127.0.0.1:443/executions').content),
    auth=HTTPBasicAuth(os.getenv('BASIC_AUTH_USERNAME'), os.getenv('BASIC_AUTH_PASSWORD')))
json.loads(res.content)

In [ ]:
json.loads(requests.get('http://127.0.0.1:443/executions').content)

In [ ]:
res = requests.get(
    f'{HOST}/reports/ib/eod/v2', 
    json=json.loads(requests.get('http://127.0.0.1:443/executions').content),
    auth=HTTPBasicAuth(os.getenv('BASIC_AUTH_USERNAME'), os.getenv('BASIC_AUTH_PASSWORD')))
json.loads(res.content)

# parse CSV report

In [ ]:
data = {}
with open('app/reports/data/U1160693_last.csv', 'r', encoding='utf-8-sig') as f:
    for row in f:
        for one_row in csv.reader([row]):
            break

        if one_row[0] != '':
            if one_row[0] in data:
                pass
            else:
                data[ one_row[0] ] = []

            data[ one_row[0] ].append(one_row)

data.keys()

In [ ]:
header = "Statement"
df_statement = pd.DataFrame(
                data=data[header][1:],
                columns=data[header][0])

df_statement = df_statement[ df_statement['Header'] == 'Data' ]
df_statement

In [ ]:
dateparser.parse("Juillet 27, 2018").date()

In [ ]:
df_statement[ df_statement['Nom champ'] == 'WhenGenerated' ]['Valeur champ'].values[0][:10]

In [ ]:
header = 'Informations instrument financier'

list_multiple_assetClasses = data[header]


dict_assetClasses = {}
k = 0
last_header = ''
for row in list_multiple_assetClasses:

    if row[1] == 'Header':
        last_header = str(k)
        k = k + 1
        dict_assetClasses[ last_header ] = []
    dict_assetClasses[ last_header ].append( row )


list_refData = []

cols = []

numeric_fields = ['Multiplicateur']

for assetClass in dict_assetClasses:
    cols = cols + dict_assetClasses[assetClass][0]
    cols = list(set(cols))

for assetClass in dict_assetClasses:
    df_refData = pd.DataFrame(
        data=dict_assetClasses[assetClass][1:],
        columns=dict_assetClasses[assetClass][0]
    )
    df_refData = df_refData[ df_refData['Header'] == 'Data' ]


    list_refData.append(df_refData)

#cols
df_refData = pd.concat(list_refData, axis=0, ignore_index=True, sort=False)

for field in numeric_fields:
    df_refData[field] = df_refData[field].replace(to_replace='--', value=np.nan)
    df_refData[field] = pd.to_numeric( df_refData[field].str.replace(',', '') )


date_fields = ['Expiration']
for field in date_fields:
    df_refData[field] = pd.to_datetime( df_refData[field], format="%Y-%m-%d" )


headers_refData = ['Catégorie d\'actifs', 'Symbole', 'Description', 'Conid', 'Multiplicateur', 'Expiration', 'Type', 'Prix de Levée' ]
headers_toShow = headers_refData


list_refData = df_refData.to_dict(orient='record')

refData = {}
for asset in list_refData:
    refData[ asset['Symbole'] ] = asset

df_refData = pd.DataFrame( list_refData )

df_refData

In [ ]:
def patch_missing_date(input_date):
    if pd.isnull(input_date):
        return None
    else:
        return input_date.strftime('%Y-%m-%d')

In [ ]:
df_refData.Expiration.apply(patch_missing_date)

In [ ]:
host = 'http://localhost:5000/'

In [ ]:
auth=HTTPBasicAuth(os.environ.get('BASIC_AUTH_USERNAME'), os.environ.get('BASIC_AUTH_PASSWORD') )

In [ ]:
requests.get(host, auth=auth )

In [ ]:
res = requests.get(host + 'reports/ib/eod/fxrate', auth=auth)
df_fxRate = pd.DataFrame( [{'Devise': 'EUR', 'Taux': 1.0}] + json.loads(res.content) )
df_fxRate.set_index('Devise', inplace=True)
df_fxRate

In [ ]:
df_fxRate.loc['EUR']['Taux']

In [ ]:
res = requests.get(host + 'reports/ib/eod/transactions', auth=auth)
df = pd.DataFrame( json.loads(res.content) )
df.head()

In [ ]:
res = requests.get('http://localhost:443/executions')
executions = json.loads(res.content)
executions

In [ ]:
res = requests.get(host + 'reports/ib/eod', auth=auth, json={'execDetails': []})
res.content

In [ ]:
res = requests.get(host + 'reports/ib/eod', auth=auth, json={'execDetails': []})
df_pos = pd.DataFrame( json.loads(res.content) )
df_pos.head()

In [ ]:
res = requests.get(host + 'reports/ib/eod', auth=auth, json=executions )
df_pos = pd.DataFrame( json.loads(res.content) )

df_pos

In [ ]:
data = {
    'execId': 'exec' + str( random.randint(1000, 10000000) ),
    'orderId': str( random.randint(1000, 10000000) ),
    'time': '20180604  13:32:52',
    'acctNumber': 'U' + str( random.randint(1000, 10000000) ),
    'exchange': 'CBOE',
    'side': 'BOT',
    'shares': 13,
    'cumQty': -13,
    'price': 7.23,
    'avgPrice': -7.23,
    'permId': str( random.randint(1000, 10000000) ),
    
    'localSymbol': 'GOOGL 6 C1200.25 Equity',
    'secType': 'OPT',
    'symbol': 'GOOGL',
    'currency': 'USD',
    'primaryExchange': '',
    'lastTradeDateOrContractMonth': '20180615',
    'multiplier': 100,
    'strike': 1200.25,
    'right': 'C'
    
}
data

In [ ]:
data2 = {
    'execId': 'exec' + str( random.randint(1000, 10000000) ),
    'orderId': str( random.randint(1000, 10000000) ),
    'time': '20180604  13:32:52',
    'acctNumber': 'U' + str( random.randint(1000, 10000000) ),
    'exchange': 'CBOE',
    'side': 'BOT',
    'shares': 13,
    'cumQty': -13,
    'price': 7.23,
    'avgPrice': -7.23,
    'permId': str( random.randint(1000, 10000000) ),
    
    'localSymbol': 'GOOGL 6 C' + str( random.randint(900, 1400) ) + '.25 Equity',
    'secType': 'OPT',
    'symbol': 'GOOGL',
    'currency': 'USD',
    'primaryExchange': '',
    'lastTradeDateOrContractMonth': '20180615',
    'multiplier': 100,
    'strike': 1200.25,
    'right': 'C'
    
}

res = requests.post(host + 'executions', auth=auth,json=data2 )
#res.content
json.loads(res.content)

In [ ]:
res = requests.get(host + 'bridge/missing', auth=auth )
#res.content
missings = json.loads(res.content)
missings['ib']

In [ ]:
for ib in missings['ib']:
    data3 = {
        "localSymbol": ib,
        "ticker": ib.upper()
    }
    #res = requests.post(host + 'bridge', auth=auth,json=data3 )
    #print(res.content)

In [ ]:
data3 = {
    'ticker': 'GOOGL US EQUITY'
}

res = requests.post(host + 'bridge/bbg/ib', auth=auth,json=data3 )
#res.content
json.loads(res.content)

In [ ]:
data4 = {
    'tickers': ['GOOGL US EQUITY', 'AAPL US equity', 'blub uq equity']
}

res = requests.post(
    f'{HOST}/bridge/bbgs/ibs', 
    auth=HTTPBasicAuth(os.getenv('BASIC_AUTH_USERNAME'), os.getenv('BASIC_AUTH_PASSWORD')), 
    json=data4 )
#res.content
json.loads(res.content)

In [ ]:
res = requests.get(host + 'bridge/backup', auth=auth )
#res.content
json.loads(res.content)

In [ ]:
data5 = {
    'tickers': ['GOOGL US EQUITY', 'AAPL US equity', 'blub uq equity']
}


res = requests.post(
    f'{HOST}/ibsymbology/bbgs/ibs', 
    auth=HTTPBasicAuth(os.getenv('BASIC_AUTH_USERNAME'), os.getenv('BASIC_AUTH_PASSWORD')), 
    json=data5 )
#res.content
json.loads(res.content)

In [ ]:
res = requests.get(
    f'{HOST}/ibsymbology/recent/2018-11-19 16:17:00', 
    auth=HTTPBasicAuth(os.getenv('BASIC_AUTH_USERNAME'), os.getenv('BASIC_AUTH_PASSWORD'))
)
#res.content
json.loads(res.content)